In [1]:
%cd ~/big_vision
import jax
import importlib
import numpy as np
from absl import logging
import jax.numpy as jnp
from jax.experimental import mesh_utils

import big_vision.utils as u
import big_vision.optax as bv_optax
import big_vision.input_pipeline as input_pipeline
from big_vision.configs.proj.image_text.siglip_replication import get_config

NamedSharding = jax.sharding.NamedSharding
P = jax.sharding.PartitionSpec
config = get_config()
model_mod = importlib.import_module(f"big_vision.models.{config.model_name}")
for m in config.get("pp_modules", ["ops_general", "ops_image", "ops_text"]): importlib.import_module(f"big_vision.pp.{m}")
def memory_stats(): return [device.memory_stats()['bytes_in_use'] for device in jax.devices()]
def info(s, *a): logging.info("\u001b[33mNOTE\u001b[0m: " + s, *a)
def write_note(note): 
	if jax.process_index() == 0: info("%s", note)

/home/austinwang/big_vision


/mnt/vlm-pd/miniconda3/envs/vlm/lib/python3.11/site-packages/IPython/core/magics/osm.py:393: UserWarning: using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


2024-04-07 23:54:43.442729: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
model = model_mod.Model(**config.get("model", {}))
train_ds, ntrain_img = input_pipeline.training(config.input)
batch_size = config.input.batch_size
total_steps = u.steps("total", config, ntrain_img, batch_size)

/mnt/vlm-pd/miniconda3/envs/vlm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Instructions for updating:
Use `tf.data.Dataset.counter(...)` instead.


Instructions for updating:
Use `tf.data.Dataset.counter(...)` instead.


In [3]:
def init(rng):
	batch = jax.tree_map(lambda x: jnp.zeros(x.shape, x.dtype.as_numpy_dtype),train_ds.element_spec)
	params = model.init(rng, batch["image"], batch["labels"])["params"]
	# Set bias in the head to a low value, such that loss is small initially.
	if "init_head_bias" in config: params["head"]["bias"] = jnp.full_like(params["head"]["bias"],config["init_head_bias"])
	return params

write_note("Inferring parameter shapes...")
rng = jax.random.PRNGKey(u.put_cpu(config.get("seed", 0)))
rng, rng_init = jax.random.split(rng)
params_shape = jax.eval_shape(init, rng_init)

write_note("Inferring optimizer state shapes...")
tx, sched_fns = bv_optax.make(config, params_shape, sched_kw=dict(total_steps=total_steps, batch_size=batch_size, data_size=ntrain_img))
opt_shape = jax.eval_shape(tx.init, params_shape)
sched_fns_cpu = [u.jit_cpu()(sched_fn) for sched_fn in sched_fns]

/tmp/ipykernel_275720/488954300.py:2: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  batch = jax.tree_map(lambda x: jnp.zeros(x.shape, x.dtype.as_numpy_dtype),train_ds.element_spec)


In [4]:
write_note("Setting up mesh...")
config_mesh = config.get("mesh", [("data", jax.device_count())])
# config_mesh = [("data", 2),('fsdp', 2)]
sharding_rules = config.get("sharding_rules", [("act_batch", "data")])
mesh_axes, mesh_size = tuple(zip(*config_mesh))
mesh_size = np.array(jax.devices()).reshape(mesh_size).shape
device_mesh = mesh_utils.create_device_mesh(mesh_size)
devices_flat = device_mesh.flatten()

In [5]:
write_note("Creating device mesh...")
mesh = jax.sharding.Mesh(device_mesh, mesh_axes)
repl_sharding = jax.sharding.NamedSharding(mesh, P())

write_note("Inferring shardings...")
train_state_shape = {"params": params_shape, "opt": opt_shape}

In [6]:
train_state_shape

{'params': {'b': ShapeDtypeStruct(shape=(1,), dtype=float32),
  'img': {'MAPHead_0': {'LayerNorm_0': {'bias': ShapeDtypeStruct(shape=(768,), dtype=float32),
     'scale': ShapeDtypeStruct(shape=(768,), dtype=float32)},
    'MlpBlock_0': {'Dense_0': {'bias': ShapeDtypeStruct(shape=(3072,), dtype=float32),
      'kernel': ShapeDtypeStruct(shape=(768, 3072), dtype=float32)},
     'Dense_1': {'bias': ShapeDtypeStruct(shape=(768,), dtype=float32),
      'kernel': ShapeDtypeStruct(shape=(3072, 768), dtype=float32)}},
    'MultiHeadDotProductAttention_0': {'key': {'bias': ShapeDtypeStruct(shape=(12, 64), dtype=float32),
      'kernel': ShapeDtypeStruct(shape=(768, 12, 64), dtype=float32)},
     'out': {'bias': ShapeDtypeStruct(shape=(768,), dtype=float32),
      'kernel': ShapeDtypeStruct(shape=(12, 64, 768), dtype=float32)},
     'query': {'bias': ShapeDtypeStruct(shape=(12, 64), dtype=float32),
      'kernel': ShapeDtypeStruct(shape=(768, 12, 64), dtype=float32)},
     'value': {'bias': Sha